# Daily Feature Pipeline
* Retrieve todays data for flights and google trends
* Add these new data to the Feature Store 

## OpenSky Recent Data 
* Use the OpenSky api to retrieve the most recent flight landing data, to update our feature group

In [16]:
import pandas as pd 
import os
import datetime
import requests 
import hopsworks

In [17]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

2025-12-30 13:05:35,393 INFO: Initializing external client
2025-12-30 13:05:35,394 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-12-30 13:05:37,144 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1296539


In [21]:
CLIENT_ID = secrets.get_secret("OPENSKY_CLIENT_ID").value
CLIENT_SECRET = secrets.get_secret("OPENSKY_CLIENT_SECRET").value
ICAO = "ESSA"

def get_access_token():
    auth_url = "https://auth.opensky-network.org/auth/realms/opensky-network/protocol/openid-connect/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET
    }
    response = requests.post(auth_url, data=data)
    response.raise_for_status()
    return response.json().get("access_token")

def fetch_yesterday_data(token):
    # Calculate yesterday's window
    yesterday = datetime.date.today() - datetime.timedelta(days=1)
    start_ts = int(datetime.datetime.combine(yesterday, datetime.time.min).timestamp())
    end_ts = int(datetime.datetime.combine(yesterday, datetime.time.max).timestamp())

    api_url = f"https://opensky-network.org/api/flights/arrival?airport={ICAO}&begin={start_ts}&end={end_ts}"
    headers = {"Authorization": f"Bearer {token}"}
    
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        flights = response.json()
        return yesterday, len(flights)
    else:
        print(f"API Error: {response.status_code}")
        return yesterday, 0

token = get_access_token()
flight_date, total_landings = fetch_yesterday_data(token)

print(f"Arlanda had {total_landings} landings on date: {flight_date}")

Arlanda had 244 landings on date: 2025-12-29


## Uploading new data to the Feature Store 

**New Flight Data**

In [23]:
new_flight_data = pd.DataFrame({
    "date": [flight_date],
    "total_landings": [total_landings],
})

In [24]:
# Retrieve feature group
fs = project.get_feature_store() 

flight_data_fg = fs.get_feature_group(
    name='flight_data_arlanda',
    version=1,
)

In [25]:
# insert new data
flight_data_fg.insert(new_flight_data, wait = True)

Uploading Dataframe: 100.00% |█| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time


Launching job: flight_data_arlanda_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1296539/jobs/named/flight_data_arlanda_1_offline_fg_materialization/executions
2025-12-30 13:15:15,998 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-12-30 13:15:19,206 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-12-30 13:15:25,595 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-12-30 13:17:05,219 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-12-30 13:17:05,387 INFO: Waiting for log aggregation to finish.
2025-12-30 13:17:14,118 INFO: Execution finished successfully.


(Job('flight_data_arlanda_1_offline_fg_materialization', 'SPARK'), None)

## Google Trends Recent Data
* Download the most recent search data for the words